In [1]:
import pandas as pd
import numpy as np
import dateutil.parser as parser

from IPython.display import display, HTML

### Defining some helper functions

In [2]:
def parse2timestamp(iso_time):
    """
    Helper function to parse an ISO-8601 formatted time to a timestamp
    """
    return parser.parse(iso_time).timestamp()

In [3]:
from math import radians, sin, cos, acos

def coord2distance (lat1, lng1, lat2, lng2, earth_radius=6371.0):
    """
    Helper function to calculate the distance between two geographic points based on their coordinates
    """
    lat1 = radians(lat1)
    lng1 = radians(lng1)
    lat2 = radians(lat2)
    lng2 = radians(lng2)
    
    return earth_radius * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng1 - lng2))

def id2coordinates (stationId, df):
    """
    Helper function to match the station or city id to its actual coordinates
    """
    lat = df.loc[df["id"] == int(stationId)]["latitude"]
    lng = df.loc[df["id"] == int(stationId)]["longitude"]

    return (lat, lng)

def trip_distance(row):
    """
    Calculate distance between stations, or if not available between respective cities, taking into account middle stations
    """
    total_distance = 0

    starting_coord = id2coordinates(row["o_station"], stations) if pd.notnull(row["o_station"]) else id2coordinates(row["o_city"], cities)
    middle_coord = [id2coordinates(x, stations) for x in row["middle_stations"].strip("{}").split(",")] if pd.notnull(row["middle_stations"]) else []
    destination_coord = id2coordinates(row["d_station"], stations) if pd.notnull(row["d_station"]) else id2coordinates(row["d_city"], cities)

    try:
        prev = starting_coord
        for mid in middle_coord:
            total_distance = total_distance + coord2distance(*prev, *mid)
            prev = mid

        total_distance = total_distance + coord2distance(*prev, *destination_coord)
    except ValueError:
        total_distance = -1
    
    return total_distance

### Loading Datasets

In [4]:
cities = pd.read_csv("./data/cities.csv")
id2city = dict(zip(cities["id"], cities["local_name"]))    # A map to match city id to its name

cities.head(10)

,id,local_name,unique_name,latitude,longitude,population
0,5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
1,76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0
2,81,"Basel, Basel-Stadt, Schweiz",basel,47.593437,7.619812,NaN
3,259,"Erlangen, Bayern, Deutschland",erlangen,49.589674,11.011961,105412.0
4,11979,"Balș, Olt, România",balș,44.353354,24.095672,NaN
5,10314,"Град Пожаревац, Централна Србија, Србија",град-пожаревац,44.619095,21.176522,NaN
6,11155,"Bussy-Lettrée, Grand-Est, France",bussy-lettree,48.804600,4.259500,NaN
7,11788,"Chamonix, Valle d'Aosta, Italia",chamonix-italia,45.817156,6.952375,NaN
8,11984,"Borger, Texas, United States of America",borger,35.667820,-101.397388,NaN
9,11825,"okres Zvolen, Banskobystrický kraj, Slovensko",okres-zvolen,48.576181,19.137116,NaN


In [5]:
stations = pd.read_csv("./data/stations.csv")
id2station = dict(zip(stations["id"], stations["unique_name"]))    # A map to match station id to its name

stations.head(10)

,id,unique_name,latitude,longitude
0,1,Aalen (Stuttgarter Straße),48.835296,10.092956
1,2,Aéroport Bordeaux-Mérignac,44.830226,-0.700883
2,3,Aéroport CDG,49.009900,2.559310
3,4,Aéroport de Berlin-Schönefeld,52.389446,13.520345
4,5,Aéroport de Dresden,51.123604,13.764737
5,6,Aéroport de Genève,46.230121,6.109288
6,7,Aéroport de Paris Beauvais-Tillé,49.462541,2.116935
7,8,Aéroport de Prague (bus station),50.107533,14.269309
8,9,Aéroport de Tegel,52.553760,13.292310
9,10,Aéroport Marco Polo,45.505432,12.338465


In [6]:
providers = pd.read_csv("./data/providers.csv")
id2provider = dict(zip(providers["id"], providers["fullname"]))    # A map to match provider id to its name

providers.head(10)

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
0,9,1,NaN,ouibus,Ouibus,True,True,True,False,bus
1,10,2,NaN,deinbus,Deinbus.de,False,False,False,False,bus
2,11,3,NaN,infobus,Infobus,False,False,False,False,bus
3,12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus
4,13,5,NaN,flixbus,Flixbus,True,False,False,False,bus
5,14,6,NaN,driivme,DriiveMe,False,False,False,False,car
6,15,8,NaN,alsa,Alsa,False,False,False,False,bus
7,16,9,NaN,distribusion,Distribusion,False,False,False,False,bus
8,17,9,31,expressbusMunchen,Expressbus Munchen,False,False,False,False,bus
9,18,9,33,berlinLinienBus,Berlin Linien Bus,False,False,False,False,bus


###  Loading Main Dataset

In this dataset, we need to calculate the time spent during each trip, and for doing so, we need to parse the given departure and arrival times represented as string to float values representing timestamps. For more informations on how this is done, see the helper function `parse2timestamp`.

Also, we need to know the distance run by each trip, and with the given informations, we can approxiate this distance using the geographic coordinates of the stations and/or the cities. For more information on how this is done, see the helper function(s) `trip_distance`.

Note 1: To limitate the computational time, we take a sub-sample from the main dataset.

Note 2: In the following procedures, NaNs are not actually an issue, so we don't need to drop them.

In [7]:
tickets = pd.read_csv("./data/ticket_data.csv")
tickets = tickets.sample(frac=1.0).reset_index(drop=True)    # Taking a small subset with index reset (in order to avoid huge computations)

# Calculating trip durations
tickets["trip_duration"] = tickets.apply(lambda row: parse2timestamp(row["arrival_ts"]) - parse2timestamp(row["departure_ts"]), axis=1)

# Trying to approximate trip distance using geographique coordinates
tickets["trip_distance_km"] = tickets.apply(trip_distance, axis=1)

tickets.head(10)

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city,trip_duration,trip_distance_km
0,6804126,8385,NaN,NaN,2017-10-06 09:00:00+00,2017-10-06 20:00:00+00,7100,2017-10-04 09:42:30.295+00,NaN,NaN,1061,126,39600.0,760.813531
1,6837498,8380,571.0,585.0,2017-10-14 10:37:00+00,2017-10-14 12:44:00+00,2980,2017-10-11 19:40:06.532+00,"{4455,4455}",{8380},185,485,7620.0,143.523381
2,6807834,8385,NaN,NaN,2017-10-06 08:00:00+00,2017-10-06 10:30:00+00,1700,2017-10-04 21:11:25.561+00,NaN,NaN,628,453,9000.0,205.842591
3,6820870,8385,NaN,NaN,2017-10-13 17:10:00+00,2017-10-13 20:50:00+00,850,2017-10-08 00:51:07.984+00,NaN,NaN,628,611,13200.0,109.663930
4,6822935,8385,NaN,NaN,2017-10-22 15:00:00+00,2017-10-22 19:00:00+00,3250,2017-10-08 14:39:34.097+00,NaN,NaN,628,562,14400.0,338.320135
5,6830679,8376,6799.0,5915.0,2017-10-11 13:52:00+00,2017-10-11 19:23:00+00,11780,2017-10-10 07:40:11.709+00,"{860,860}",{8380},628,348,19860.0,685.535176
6,6825243,8385,NaN,NaN,2017-10-09 15:30:14+00,2017-10-09 20:20:14+00,3500,2017-10-08 18:29:41.199+00,NaN,NaN,628,485,17400.0,390.628263
7,6850559,8385,NaN,NaN,2017-10-20 13:00:00+00,2017-10-20 15:50:00+00,1550,2017-10-15 14:16:11.032+00,NaN,NaN,628,453,10200.0,205.842591
8,6845992,8371,10642.0,10014.0,2017-12-22 14:45:00+00,2017-12-24 08:00:00+00,9390,2017-10-14 07:12:47.165+00,"{500,245}",{13},628,7507,148500.0,1076.074089
9,6868341,8380,279.0,747.0,2017-10-28 12:39:00+00,2017-10-28 17:55:00+00,6000,2017-10-19 07:44:39.897+00,"{5758,5758}",{8380},628,488,18960.0,337.796114


### Grouping the trips
We group the trips by their origine station and destination station.

We do this in order to calculate statistics (such as min, max, and average) about price and trip duration.

Notice that stations for some trips are not indicated (NaNs) and thus those trips are not included in the survey. A solution for this would be to group by cities instead of stations.

In [8]:
grouped_by_station = tickets.groupby(["o_station", "d_station"])
# grouped_by_city = tickets.groupby(["o_city", "d_city"])

#### Statistics about trip price

In [9]:
for (start_point, end_point), group in grouped_by_station:
    
    # The stats
    min_price = group["price_in_cents"].min()
    max_price = group["price_in_cents"].max()
    avg_price = group["price_in_cents"].mean()

    # Creating a new dataframe holding the stats
    trip_information = pd.DataFrame([{
        "Departure station": id2station[start_point],
        "Arrival station": id2station[end_point],
        "Lowest bid (€)": min_price / 100,
        "Highest bid (€)": max_price / 100,
        "Average bid (€)": avg_price / 100
    }])
    trip_information = trip_information[["Departure station", "Arrival station", "Lowest bid (€)", "Highest bid (€)", "Average bid (€)"]]
    
    # Arranging the display
    display(HTML("<h3>From: {}".format(id2station[start_point])))
    display(HTML("<h3>To: {}".format(id2station[end_point])))
    display(trip_information)
    display(HTML("<h4>You can choose your trip in the list below :"))
    
    # Extracting a sub-dataframe holding trips with lowest bids for each couple (o_station, d_station)
    # Notice that some transformations are done to make the display human-friendly
    min_group = group[group["price_in_cents"] == min_price]
    min_group.loc[:, ("o_station")] = min_group["o_station"].apply(lambda x: id2station[x])
    min_group.loc[:, ("o_city")] = min_group["o_city"].apply(lambda x: id2city[x])
    min_group.loc[:, ("d_station")] = min_group["d_station"].apply(lambda x: id2station[x])
    min_group.loc[:, ("d_city")] = min_group["d_city"].apply(lambda x: id2city[x])
    min_group.loc[:, ("company")] = min_group["company"].apply(lambda x: id2provider[x])
    min_group.loc[:, ("other_companies")] = min_group["other_companies"].apply(lambda x: [id2provider[int(y)] for y in x.strip("{}").split(",")])
    min_group.loc[:, ("middle_stations")] = min_group["middle_stations"].apply(lambda x: [id2station[int(y)] for y in x.strip("{}").split(",")])

    min_group = min_group[["o_station", "o_city", "d_station", "d_city", "price_in_cents", "departure_ts", "arrival_ts", "company", "trip_distance_km", "other_companies", "middle_stations"]]
    min_group = min_group.head(5)
    
    display(min_group)
    
    print("=" * 100)
    print("=" * 100)

,Departure station,Arrival station,Lowest bid (€),Highest bid (€),Average bid (€)
0,Aéroport CDG,Gare de Liège-Guillemins,27.9,27.9,27.9


,o_station,o_city,d_station,d_city,price_in_cents,departure_ts,arrival_ts,company,trip_distance_km,other_companies,middle_stations
130,Aéroport CDG,"Paris, Île-de-France, France",Gare de Liège-Guillemins,"Liège, Wallonie, Belgique",2790,2017-10-21 17:40:00+00,2017-10-22 16:15:00+00,Flixbus,360.644161,[Deinbus.de],"[Gare Lille-Europe, Gare Lille-Europe]"


,Departure station,Arrival station,Lowest bid (€),Highest bid (€),Average bid (€)
0,Aéroport CDG,Gare routière Aéroport Nice Terminal 2,58.8,58.8,58.8


,o_station,o_city,d_station,d_city,price_in_cents,departure_ts,arrival_ts,company,trip_distance_km,other_companies,middle_stations
487,Aéroport CDG,"Paris, Île-de-France, France",Gare routière Aéroport Nice Terminal 2,"Nice, Provence-Alpes-Côte d'Azur, France",5880,2017-10-20 00:50:00+00,2017-10-20 16:45:00+00,Flixbus,699.496681,[Flixbus],"[Gare SNCF Lyon-Perrache, Gare SNCF Lyon-Perra..."


,Departure station,Arrival station,Lowest bid (€),Highest bid (€),Average bid (€)
0,Aéroport CDG,Gare routière de Niort,20.0,20.0,20.0


...

,o_station,o_city,d_station,d_city,price_in_cents,departure_ts,arrival_ts,company,trip_distance_km,other_companies,middle_stations
35,Gare de l'Est,"Paris, Île-de-France, France",Saint-Dizier,"Saint-Dizier, Grand-Est, France",6180,2017-10-14 09:28:00+00,2017-10-14 14:19:00+00,TGV,218.391834,[TER],"[Champagne-Ardenne TGV, Champagne-Ardenne TGV]"


,Departure station,Arrival station,Lowest bid (€),Highest bid (€),Average bid (€)
0,"Tourcoing (Bus stop on ""Avenue Salvador Allende"")",Rouen Rive Droite,47.0,47.0,47.0


,o_station,o_city,d_station,d_city,price_in_cents,departure_ts,arrival_ts,company,trip_distance_km,other_companies,middle_stations
0,"Tourcoing (Bus stop on ""Avenue Salvador Allende"")","Tourcoing, Hauts-de-France, France",Rouen Rive Droite,"Rouen, Normandie, France",4700,2017-10-19 06:10:00+00,2017-10-19 10:00:00+00,TGV,324.701956,[Intercités],"[Gare du Nord, Paris Saint-Lazare]"


,Departure station,Arrival station,Lowest bid (€),Highest bid (€),Average bid (€)
0,Paris - Bibliothèque François Mitterrand,Gare de Châlons-en-Champagne,68.0,68.0,68.0


,o_station,o_city,d_station,d_city,price_in_cents,departure_ts,arrival_ts,company,trip_distance_km,other_companies,middle_stations
328,Paris - Bibliothèque François Mitterrand,"Paris, Île-de-France, France",Gare de Châlons-en-Champagne,"Châlons-en-Champagne, Grand-Est, France",6800,2017-10-11 12:05:00+00,2017-10-11 17:10:00+00,Vatry,155.552001,[Vatry],"[Vatry, Vatry]"


#### Statistics about trip duration

In [10]:
for (start_point, end_point), group in grouped_by_station:
    
    # The stats
    min_duration = group["trip_duration"].min()
    max_duration = group["trip_duration"].max()
    avg_duration = group["trip_duration"].mean()

    # Creating a new dataframe holding the stats
    trip_information = pd.DataFrame([{
        "Departure station": id2station[start_point],
        "Arrival station": id2station[end_point],
        "Shortest trip (minutes)": min_duration / 60,
        "Longest trip (minutes)": max_duration / 60,
        "Average trip (minutes)": avg_duration / 60
    }])
    trip_information = trip_information[["Departure station", "Arrival station", "Shortest trip (minutes)" ,"Longest trip (minutes)" ,"Average trip (minutes)"]]

    # Arranging the display
    display(HTML("<h3>From: {}".format(id2station[start_point])))
    display(HTML("<h3>To: {}".format(id2station[end_point])))
    display(trip_information)
    display(HTML("<h4>You can choose your trip in the list below :"))
    
    # Extracting a sub-dataframe holding trips with shortest duration for each couple (o_station, d_station)
    # Notice that some transformations are done to make the display human-friendly
    min_group = group[group["trip_duration"] == min_duration]
    min_group.loc[:, ("o_station")] = min_group["o_station"].apply(lambda x: id2station[x])
    min_group.loc[:, ("o_city")] = min_group["o_city"].apply(lambda x: id2city[x])
    min_group.loc[:, ("d_station")] = min_group["d_station"].apply(lambda x: id2station[x])
    min_group.loc[:, ("d_city")] = min_group["d_city"].apply(lambda x: id2city[x])
    min_group.loc[:, ("company")] = min_group["company"].apply(lambda x: id2provider[x])
    min_group.loc[:, ("other_companies")] = min_group["other_companies"].apply(lambda x: [id2provider[int(y)] for y in x.strip("{}").split(",")])
    min_group.loc[:, ("middle_stations")] = min_group["middle_stations"].apply(lambda x: [id2station[int(y)] for y in x.strip("{}").split(",")])

    min_group = min_group[["o_station", "o_city", "d_station", "d_city", "price_in_cents", "departure_ts", "arrival_ts", "company", "other_companies", "middle_stations"]]
    min_group = min_group.head(5)
    
    display(min_group)
    
    print("=" * 100)
    print("=" * 100)

,Departure station,Arrival station,Shortest trip (minutes),Longest trip (minutes),Average trip (minutes)
0,Aéroport CDG,Gare de Liège-Guillemins,1355.0,1355.0,1355.0


,o_station,o_city,d_station,d_city,price_in_cents,departure_ts,arrival_ts,company,other_companies,middle_stations
130,Aéroport CDG,"Paris, Île-de-France, France",Gare de Liège-Guillemins,"Liège, Wallonie, Belgique",2790,2017-10-21 17:40:00+00,2017-10-22 16:15:00+00,Flixbus,[Deinbus.de],"[Gare Lille-Europe, Gare Lille-Europe]"


,Departure station,Arrival station,Shortest trip (minutes),Longest trip (minutes),Average trip (minutes)
0,Aéroport CDG,Gare routière Aéroport Nice Terminal 2,955.0,955.0,955.0


,o_station,o_city,d_station,d_city,price_in_cents,departure_ts,arrival_ts,company,other_companies,middle_stations
487,Aéroport CDG,"Paris, Île-de-France, France",Gare routière Aéroport Nice Terminal 2,"Nice, Provence-Alpes-Côte d'Azur, France",5880,2017-10-20 00:50:00+00,2017-10-20 16:45:00+00,Flixbus,[Flixbus],"[Gare SNCF Lyon-Perrache, Gare SNCF Lyon-Perra..."


,Departure station,Arrival station,Shortest trip (minutes),Longest trip (minutes),Average trip (minutes)
0,Aéroport CDG,Gare routière de Niort,595.0,595.0,595.0


...

,o_station,o_city,d_station,d_city,price_in_cents,departure_ts,arrival_ts,company,other_companies,middle_stations
35,Gare de l'Est,"Paris, Île-de-France, France",Saint-Dizier,"Saint-Dizier, Grand-Est, France",6180,2017-10-14 09:28:00+00,2017-10-14 14:19:00+00,TGV,[TER],"[Champagne-Ardenne TGV, Champagne-Ardenne TGV]"


,Departure station,Arrival station,Shortest trip (minutes),Longest trip (minutes),Average trip (minutes)
0,"Tourcoing (Bus stop on ""Avenue Salvador Allende"")",Rouen Rive Droite,230.0,230.0,230.0


,o_station,o_city,d_station,d_city,price_in_cents,departure_ts,arrival_ts,company,other_companies,middle_stations
0,"Tourcoing (Bus stop on ""Avenue Salvador Allende"")","Tourcoing, Hauts-de-France, France",Rouen Rive Droite,"Rouen, Normandie, France",4700,2017-10-19 06:10:00+00,2017-10-19 10:00:00+00,TGV,[Intercités],"[Gare du Nord, Paris Saint-Lazare]"


,Departure station,Arrival station,Shortest trip (minutes),Longest trip (minutes),Average trip (minutes)
0,Paris - Bibliothèque François Mitterrand,Gare de Châlons-en-Champagne,305.0,305.0,305.0


,o_station,o_city,d_station,d_city,price_in_cents,departure_ts,arrival_ts,company,other_companies,middle_stations
328,Paris - Bibliothèque François Mitterrand,"Paris, Île-de-France, France",Gare de Châlons-en-Champagne,"Châlons-en-Champagne, Grand-Est, France",6800,2017-10-11 12:05:00+00,2017-10-11 17:10:00+00,Vatry,[Vatry],"[Vatry, Vatry]"


### Extracting statistics about average travel cost and duration for some distance ranges and means of transport

For this purpose, we need to split the main dataframe with respect to different range values (200 km, 800 km, ...), and with respect to transport means (bus, train, ...)

In [9]:
transport_types = providers["transport_type"].unique()    # We need to know what are the available means of transport
transport_types

array(['bus', 'car', 'train', 'carpooling'], dtype=object)

In [10]:
# Splitting the providers with regard to transport type
bus_providers = providers[providers["transport_type"] == "bus"]["id"]
car_providers = providers[providers["transport_type"] == "car"]["id"]
train_providers = providers[providers["transport_type"] == "train"]["id"]
carpooling_providers = providers[providers["transport_type"] == "carpooling"]["id"]

# Splitting the trips with regard to trip distance
short_trips = tickets[tickets["trip_distance_km"] < 200]
medium_trips = tickets[(tickets["trip_distance_km"] >= 200) & (tickets["trip_distance_km"] < 800)]
long_trips = tickets[(tickets["trip_distance_km"] >= 800) & (tickets["trip_distance_km"] < 2000)]
extra_long_trips = tickets[tickets["trip_distance_km"] >= 2000]

assert(short_trips.size + medium_trips.size + long_trips.size + extra_long_trips.size == tickets.size)


# Defining what are the columns we need to display
cols = pd.MultiIndex.from_product([transport_types, ["average_price_cents", "average_duration"]])
cols = cols.insert(0, "trip_range")
cols = cols.insert(1, "total")

# Creating a new dataframe holding the stats
statistics = pd.DataFrame.from_records([
    (
        "0-200 km",
        short_trips.size,
        short_trips.loc[short_trips["company"].isin(bus_providers), "price_in_cents"].mean() / 100,
        short_trips.loc[short_trips["company"].isin(bus_providers), "trip_duration"].mean() / 60,
        short_trips.loc[short_trips["company"].isin(car_providers), "price_in_cents"].mean() / 100,
        short_trips.loc[short_trips["company"].isin(car_providers), "trip_duration"].mean() / 60,
        short_trips.loc[short_trips["company"].isin(train_providers), "price_in_cents"].mean() / 100,
        short_trips.loc[short_trips["company"].isin(train_providers), "trip_duration"].mean() / 60,
        short_trips.loc[short_trips["company"].isin(carpooling_providers), "price_in_cents"].mean() / 100,
        short_trips.loc[short_trips["company"].isin(carpooling_providers), "trip_duration"].mean() / 60
    ), (
        "200-800 km",
        medium_trips.size,
        medium_trips.loc[medium_trips["company"].isin(bus_providers), "price_in_cents"].mean() / 100,
        medium_trips.loc[medium_trips["company"].isin(bus_providers), "trip_duration"].mean() / 60,
        medium_trips.loc[medium_trips["company"].isin(car_providers), "price_in_cents"].mean() / 100,
        medium_trips.loc[medium_trips["company"].isin(car_providers), "trip_duration"].mean() / 60,
        medium_trips.loc[medium_trips["company"].isin(train_providers), "price_in_cents"].mean() / 100,
        medium_trips.loc[medium_trips["company"].isin(train_providers), "trip_duration"].mean() / 60,
        medium_trips.loc[medium_trips["company"].isin(carpooling_providers), "price_in_cents"].mean() / 100,
        medium_trips.loc[medium_trips["company"].isin(carpooling_providers), "trip_duration"].mean() / 60
    ), (
        "800-2000 km",
        long_trips.size,
        long_trips.loc[long_trips["company"].isin(bus_providers), "price_in_cents"].mean() / 100,
        long_trips.loc[long_trips["company"].isin(bus_providers), "trip_duration"].mean() / 60,
        long_trips.loc[long_trips["company"].isin(car_providers), "price_in_cents"].mean() / 100,
        long_trips.loc[long_trips["company"].isin(car_providers), "trip_duration"].mean() / 60,
        long_trips.loc[long_trips["company"].isin(train_providers), "price_in_cents"].mean() / 100,
        long_trips.loc[long_trips["company"].isin(train_providers), "trip_duration"].mean() / 60,
        long_trips.loc[long_trips["company"].isin(carpooling_providers), "price_in_cents"].mean() / 100,
        long_trips.loc[long_trips["company"].isin(carpooling_providers), "trip_duration"].mean() / 60
    ), (
        "2000+ km",
        extra_long_trips.size,
        extra_long_trips.loc[extra_long_trips["company"].isin(bus_providers), "price_in_cents"].mean() / 100,
        extra_long_trips.loc[extra_long_trips["company"].isin(bus_providers), "trip_duration"].mean() / 60,
        extra_long_trips.loc[extra_long_trips["company"].isin(car_providers), "price_in_cents"].mean() / 100,
        extra_long_trips.loc[extra_long_trips["company"].isin(car_providers), "trip_duration"].mean() / 60,
        extra_long_trips.loc[extra_long_trips["company"].isin(train_providers), "price_in_cents"].mean() / 100,
        extra_long_trips.loc[extra_long_trips["company"].isin(train_providers), "trip_duration"].mean() / 60,
        extra_long_trips.loc[extra_long_trips["company"].isin(carpooling_providers), "price_in_cents"].mean() / 100,
        extra_long_trips.loc[extra_long_trips["company"].isin(carpooling_providers), "trip_duration"].mean() / 60
    )
], columns=cols)


statistics

trip_range   total                 bus                   \
                       average_price_cents average_duration   
0     0-200 km  183680           21.619639       471.551724   
1   200-800 km  825818           33.927431       888.205419   
2  800-2000 km   28854           68.125959      1630.711129   
3     2000+ km       0                 NaN              NaN   

                  car                                train                   \
  average_price_cents average_duration average_price_cents average_duration   
0                 NaN              NaN           42.891207       342.822123   
1                 NaN              NaN           88.315281       443.099367   
2                 NaN              NaN          166.053456       734.804916   
3                 NaN              NaN                 NaN              NaN   

           carpooling                   
  average_price_cents average_duration  
0           11.773524       117.122374  
1           32.180643       285.787874  
2           86.265455       786.013986  
3                 NaN              NaN